# Pescador demo

This notebook illustrates some of the basic functionality of [pescador](https://github.com/bmcfee/pescador): a package to facilitate iterative learning from data streams (implemented as python generators).

In [1]:
import pescador

import numpy as np
np.set_printoptions(precision=4)
import sklearn
import sklearn.datasets
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.metrics

In [2]:
def data_generator(X, Y, m=20, scale = 1e-1):
    '''A gaussian noise generator for data
    
    Parameters
    ----------
    X : ndarray
        features, n_samples by dimensions
        
    Y : ndarray
        labels, n_samples
        
    m : int
        size of the minibatches to generate
        
    scale : float > 0
        scale of the noise to add
        
    Generates
    ---------
    batch
        An infinite stream of batch dictionaries
        batch = dict(X=X[i], Y=Y[i])
    '''
    
    X = np.atleast_2d(X)
    Y = np.atleast_1d(Y)

    
    n, d = X.shape
    
    while True:
        i = np.random.randint(0, n, size=m)
        
        noise = scale * np.random.randn(m, d)
        
        yield {'X': X[i] + noise, 'Y': Y[i]}

In [3]:
# Load up the iris dataset for the demo
data = sklearn.datasets.load_iris()
X, Y = data.data, data.target
classes = np.unique(Y)

In [5]:
# What does the data stream look like?

# First, we'll wrap the generator function in a Streamer object.
# This is necessary for a few reasons, notably so that we can re-instantiate
# the generator multiple times (eg once per epoch)

stream = pescador.Streamer(data_generator, X, Y)

# The buffer_batch() function takes a batch stream as input, and
# carves it into batches of up to buffer_size (3, in this case) samples
# the buffer size can be larger or smaller than the native size of the input batches
for q in pescador.buffer_batch(stream.generate(max_batches=1), 3):
    print q

{'Y': array([1, 2, 0]), 'X': array([[ 5.8958,  3.2539,  4.7756,  1.9052],
       [ 6.376 ,  2.9074,  4.866 ,  1.8192],
       [ 4.3881,  3.0593,  1.3687,  0.3059]])}
{'Y': array([2, 2, 0]), 'X': array([[ 6.7266,  2.4949,  5.9466,  1.7307],
       [ 5.8329,  2.696 ,  5.0676,  1.8783],
       [ 5.4087,  3.6965,  1.3133,  0.2138]])}
{'Y': array([0, 0, 2]), 'X': array([[ 5.1651,  3.5178,  1.4245,  0.0344],
       [ 4.5477,  3.3718,  1.413 ,  0.2002],
       [ 6.2034,  2.895 ,  5.6942,  1.8306]])}
{'Y': array([1, 0, 2]), 'X': array([[ 5.7593,  2.6121,  4.1147,  1.2895],
       [ 5.0935,  3.5156,  1.5837,  0.4479],
       [ 6.3018,  3.2887,  5.509 ,  2.3479]])}
{'Y': array([0, 1, 2]), 'X': array([[ 4.8191,  3.3147,  1.7729,  0.0847],
       [ 5.5358,  2.2858,  3.9109,  1.3077],
       [ 6.4131,  3.3305,  5.097 ,  2.0368]])}
{'Y': array([0, 0, 1]), 'X': array([[ 4.3614,  3.2456,  1.3656,  0.0904],
       [ 4.8729,  3.1725,  1.2827,  0.1916],
       [ 5.951 ,  3.1406,  4.6995,  1.7543]])}
{'Y'

# Benchmarking
We can benchmark our learner's efficiency by running a couple of experiments on the Iris dataset.

Our classifier will be L1-regularized logistic regression.

In [7]:
%%time
for train, test in sklearn.cross_validation.ShuffleSplit(len(X),
                                                         n_iter=2,
                                                         test_size=0.2):
    
    # Make an SGD learner, nothing fancy here
    classifier = sklearn.linear_model.SGDClassifier(verbose=0, 
                                                    loss='log',
                                                    penalty='l1', 
                                                    n_iter=1)
    
    # Make a streamable wrapper
    model = pescador.StreamLearner(classifier)
    
    # Again, build a streamer object
    stream = pescador.Streamer(data_generator, X[train], Y[train])
    
    # we'll buffer into batches of 16 samples each
    samples = pescador.buffer_batch(stream.generate(max_batches=5e3//20),
                                    20)
    
    # And train the model on the stream.
    # iter_fit() works just like partial_fit(), except that the input is a generator.
    model.iter_fit(samples, classes=classes)
    
    # How's it do on the test set?
    print 'Test-set accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(Y[test], model.predict(X[test])))
    print '# Steps: ' + str(model.estimator.t_)

Test-set accuracy: 0.733
# Steps: 5001.0
Test-set accuracy: 0.867
# Steps: 5001.0
CPU times: user 195 ms, sys: 2.08 ms, total: 197 ms
Wall time: 198 ms


# Parallelism

It's possible that the learner is more or less efficient than the data generator.  If the data generator has higher latency than the learner (SGDClassifier), then this will slow down the learning.

Pescador uses zeromq to parallelize data stream generation, effectively decoupling it from the learner.

In [8]:
%%time
for train, test in sklearn.cross_validation.ShuffleSplit(len(X), n_iter=2, test_size=0.2):
    
    # Make an SGD learner, nothing fancy here
    classifier = sklearn.linear_model.SGDClassifier(verbose=0, 
                                                    loss='log',
                                                    penalty='l1', 
                                                    n_iter=1)
    
    # Make a streamable wrapper
    model = pescador.StreamLearner(classifier)
    
    # First, turn the data_generator function into a Streamer object
    stream = pescador.Streamer(data_generator, X[train], Y[train])
    
    # Then, send this thread to a second process
    zmq_stream = pescador.zmq_stream(5156, stream, max_batches=5e3//20)
    
    # Run the output through a second buffer for mini-batch training
    samples = pescador.buffer_batch(zmq_stream, 20)
    
    # And fit on the stream
    model.iter_fit(samples, classes=classes)
    
    # How's it do on the test set?
    print 'Test-set accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(Y[test], model.predict(X[test])))
    print '# Steps: ' + str(model.estimator.t_)

TypeError: zmq_stream() got an unexpected keyword argument 'max_batches'